# Interactive Wave Simulation - String Theory Inspired

This notebook provides interactive controls to explore wave dynamics with forces derived from a Lennard-Jones-like potential.

## Features:
- Real-time parameter adjustment with sliders
- Animated wave propagation
- 3D visualization of wave evolution
- Energy monitoring
- Phase space exploration

In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed, FloatSlider, IntSlider, HBox, VBox, Button, Output
import ipywidgets as widgets
from IPython.display import display

# Import our modules
from solver import wave_solver, potential_function, force_function
from interactive_visualization import (
    create_animated_wave,
    create_3d_wave_surface,
    create_interactive_potential_force,
    create_energy_monitor,
    create_phase_space
)

## 1. Interactive Potential and Force Explorer

Adjust the force constants K1 and K2 to see how they affect the potential and force functions.

In [ ]:
def plot_potential_force(K1_exp, K2_exp, u_min, u_max):
    """
    Plot potential and force functions with adjustable parameters.
    
    Args:
        K1_exp: Log10 of K1 (repulsive constant)
        K2_exp: Log10 of K2 (attractive constant)
        u_min: Minimum displacement to plot
        u_max: Maximum displacement to plot
    """
    K1 = 10**K1_exp
    K2 = 10**K2_exp
    
    u_range = np.linspace(u_min, u_max, 500)
    potential = potential_function(u_range, K1, K2)
    force = force_function(u_range, K1, K2)
    
    fig = create_interactive_potential_force(u_range, potential, force,
                                            title=f"Potential & Force (K1=10^{K1_exp:.1f}, K2=10^{K2_exp:.1f})")
    fig.show()

# Create interactive widget
interact(
    plot_potential_force,
    K1_exp=FloatSlider(value=35, min=30, max=40, step=0.5, description='log10(K1)', continuous_update=False),
    K2_exp=FloatSlider(value=18, min=15, max=22, step=0.5, description='log10(K2)', continuous_update=False),
    u_min=FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='u min', continuous_update=False),
    u_max=FloatSlider(value=5.0, min=1.0, max=10.0, step=0.5, description='u max', continuous_update=False)
);

## 2. Interactive Wave Simulation

Run a complete wave simulation with adjustable parameters and see the animated results.

In [ ]:
class WaveSimulationWidget:
    def __init__(self):
        # Create parameter sliders
        self.K1_slider = FloatSlider(value=35, min=30, max=40, step=0.5, description='log10(K1)')
        self.K2_slider = FloatSlider(value=18, min=15, max=22, step=0.5, description='log10(K2)')
        self.c_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Wave Speed (c)')
        self.dt_slider = FloatSlider(value=0.01, min=0.001, max=0.05, step=0.001, description='Time Step (dt)')
        self.dx_slider = FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='Space Step (dx)')
        self.total_time_slider = FloatSlider(value=5.0, min=1.0, max=20.0, step=1.0, description='Total Time')
        
        # Initial condition parameters
        self.amplitude_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Amplitude')
        self.center_slider = FloatSlider(value=25.0, min=10.0, max=40.0, step=1.0, description='Center Position')
        self.width_slider = FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='Gaussian Width')
        
        # Visualization type selector
        self.viz_type = widgets.Dropdown(
            options=['Animated 2D', '3D Surface', 'Energy Monitor', 'Phase Space'],
            value='Animated 2D',
            description='Visualization:'
        )
        
        # Run button
        self.run_button = Button(description='Run Simulation', button_style='success')
        self.run_button.on_click(self.run_simulation)
        
        # Output area
        self.output = Output()
        
        # Layout
        self.physics_params = VBox([
            widgets.HTML("<h3>Physics Parameters</h3>"),
            self.K1_slider,
            self.K2_slider,
            self.c_slider
        ])
        
        self.numerical_params = VBox([
            widgets.HTML("<h3>Numerical Parameters</h3>"),
            self.dt_slider,
            self.dx_slider,
            self.total_time_slider
        ])
        
        self.initial_conditions = VBox([
            widgets.HTML("<h3>Initial Conditions</h3>"),
            self.amplitude_slider,
            self.center_slider,
            self.width_slider
        ])
        
        self.controls = HBox([self.physics_params, self.numerical_params, self.initial_conditions])
        self.ui = VBox([self.controls, self.viz_type, self.run_button, self.output])
    
    def run_simulation(self, button):
        with self.output:
            self.output.clear_output(wait=True)
            print("Running simulation...")
            
            # Get parameters
            K1 = 10**self.K1_slider.value
            K2 = 10**self.K2_slider.value
            c = self.c_slider.value
            dt = self.dt_slider.value
            dx = self.dx_slider.value
            total_time = self.total_time_slider.value
            
            amplitude = self.amplitude_slider.value
            center = self.center_slider.value
            width = self.width_slider.value
            
            # Create spatial grid
            x = np.arange(0, 50, dx)
            
            # Initial conditions
            u0 = amplitude * np.exp(-((x - center) / width)**2)
            u0_prev = u0.copy()
            
            # Run simulation
            try:
                u_history, time_points = wave_solver(
                    x, u0, u0_prev, c, dt, total_time, K1, K2
                )
                
                print(f"Simulation complete! {len(u_history)} time steps computed.")
                
                # Create visualization based on selected type
                viz_type = self.viz_type.value
                
                if viz_type == 'Animated 2D':
                    # Sample frames for animation (max 100 frames)
                    step = max(1, len(u_history) // 100)
                    u_sampled = u_history[::step]
                    t_sampled = time_points[::step]
                    fig = create_animated_wave(x, u_sampled, t_sampled)
                    
                elif viz_type == '3D Surface':
                    # Sample for 3D (max 50 time slices)
                    step = max(1, len(u_history) // 50)
                    u_sampled = u_history[::step]
                    t_sampled = time_points[::step]
                    fig = create_3d_wave_surface(x, u_sampled, t_sampled)
                    
                elif viz_type == 'Energy Monitor':
                    # Calculate energies
                    kinetic_energy = []
                    potential_energy = []
                    
                    for u in u_history:
                        # Kinetic energy (approximation using finite differences)
                        ke = 0.5 * np.sum(u**2) * dx
                        kinetic_energy.append(ke)
                        
                        # Potential energy
                        pe = np.sum(potential_function(np.abs(u) + 0.1, K1, K2)) * dx
                        potential_energy.append(pe)
                    
                    kinetic_energy = np.array(kinetic_energy)
                    potential_energy = np.array(potential_energy)
                    total_energy = kinetic_energy + potential_energy
                    
                    fig = create_energy_monitor(time_points, kinetic_energy, 
                                               potential_energy, total_energy)
                    
                elif viz_type == 'Phase Space':
                    # Use center point for phase space
                    center_idx = len(x) // 2
                    position = [u[center_idx] for u in u_history]
                    
                    # Calculate velocity using finite differences
                    velocity = np.gradient(position, dt)
                    
                    fig = create_phase_space(position, velocity, time_points,
                                            title=f"Phase Space at x={x[center_idx]:.2f}")
                
                fig.show()
                
            except Exception as e:
                print(f"Error during simulation: {e}")
                import traceback
                traceback.print_exc()
    
    def display(self):
        display(self.ui)

# Create and display the widget
sim_widget = WaveSimulationWidget()
sim_widget.display()

## 3. Multi-Scenario Comparison

Compare different initial conditions or parameter sets side-by-side.

In [ ]:
def compare_scenarios(scenario1_amplitude, scenario2_amplitude, scenario3_amplitude):
    """
    Compare three different scenarios with varying initial amplitudes.
    """
    from plotly.subplots import make_subplots
    
    # Common parameters
    K1 = 1e35
    K2 = 1e18
    c = 1.0
    dt = 0.01
    dx = 0.1
    total_time = 5.0
    
    x = np.arange(0, 50, dx)
    amplitudes = [scenario1_amplitude, scenario2_amplitude, scenario3_amplitude]
    colors = ['blue', 'red', 'green']
    
    fig = make_subplots(rows=1, cols=3,
                       subplot_titles=[f'Amplitude = {a}' for a in amplitudes])
    
    for i, (amp, color) in enumerate(zip(amplitudes, colors), 1):
        # Initial condition
        u0 = amp * np.exp(-((x - 25) / 2.0)**2)
        u0_prev = u0.copy()
        
        # Run simulation
        u_history, time_points = wave_solver(x, u0, u0_prev, c, dt, total_time, K1, K2)
        
        # Plot final state
        fig.add_trace(
            go.Scatter(x=x, y=u_history[-1], name=f'Final (A={amp})',
                      line=dict(color=color, width=2)),
            row=1, col=i
        )
        
        # Plot initial state (dashed)
        fig.add_trace(
            go.Scatter(x=x, y=u0, name=f'Initial (A={amp})',
                      line=dict(color=color, width=1, dash='dash'),
                      showlegend=(i==1)),
            row=1, col=i
        )
    
    fig.update_layout(height=400, title_text="Scenario Comparison",
                     template='plotly_white')
    fig.show()

# Create interactive comparison
interact(
    compare_scenarios,
    scenario1_amplitude=FloatSlider(value=0.5, min=0.1, max=3.0, step=0.1, description='Scenario 1'),
    scenario2_amplitude=FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Scenario 2'),
    scenario3_amplitude=FloatSlider(value=2.0, min=0.1, max=3.0, step=0.1, description='Scenario 3')
);

## 4. Custom Experiment

Use this cell to create your own custom experiments and visualizations!

In [ ]:
# Your custom code here
